read data
data needs to be sorted by skill, student id, time
sort if not sorted already

In [1]:
import numpy as np
import pandas as pd
import ctypes as ct
import sklearn.metrics as sk
import random
import statistics

df = pd.read_excel('/Users/nabilch/Desktop/bkt_c/rori_rising_data_for_bkt_microlesson.xlsx')

get parameters for each skill

In [2]:
def get_parmtrs_4_1_skill(temp_df):
    size=len(temp_df)
    right=(ct.c_int*size)()
    student_id=(ct.c_int*size)()
    rows=(ct.c_int)()
    rows=size
    previous_student=""
    student_count=0
    for i in range(size):
        right[i]=temp_df['right'][i]
        if(temp_df['student'][i]!=previous_student):
            student_count=student_count+1
            previous_student=temp_df['student'][i]
        student_id[i]=student_count
        
    libObj.get_bkt_par_c.restype = ct.POINTER(ct.c_float)
    parameters=libObj.get_bkt_par_c(student_id, right, rows)
    L0_temp=round(parameters[0],3)
    T_temp=round(parameters[1],3)
    S_temp=round(parameters[2],3)
    G_temp=round(parameters[3],3)
    temp=libObj.free_memory(parameters)
    return L0_temp,T_temp,S_temp,G_temp

get parameters for all skills
only considering skills for which at least 25 students attempted at least 3 questions
can turn this check (block of code) off if wanted

In [3]:
def get_parmtrs_all_skills(df_temp):
    all_skills_parmtrs=pd.DataFrame(columns=['skill','L0','T','S','G'])
    skills=df_temp.skill.unique()
    for s in skills:
        df_skill=df_temp[df_temp['skill']==s].reset_index(drop=True)
        # checcking if there are at least 25 students with at least 3 qns each
        temp=df_skill.student.value_counts()
        if (len(temp)<25):
            continue
        if (temp[24]<3):
            continue
        L0,T,S,G=get_parmtrs_4_1_skill(df_skill)
        all_skills_parmtrs=all_skills_parmtrs._append({'skill':s,'L0':L0,'T':T,'S':S,'G':G},ignore_index=True)
    return all_skills_parmtrs

get predicted probabilities based on model parameters
only for skills that were present in training data

In [4]:
def get_predictions(temp_df, temp_par):
    predictions=temp_df[['student','skill','right']]
    # only for skills for which we have bkt parameters
    predictions=predictions[predictions['skill'].isin(temp_par['skill'])]
    predictions=predictions.merge(temp_par, on='skill', how='left')
    previous_student=''
    previous_skill=''
    for i in range(len(predictions)):
        if((predictions['student'][i]==previous_student) and (predictions['skill'][i]==previous_skill)):
            predictions.at[i,'PriorL']=PostL
        else:
            predictions.at[i,'PriorL']=predictions['L0'][i]
        
        if(predictions['right'][i]==0):
            cond_prob=(predictions['PriorL'][i]*predictions['S'][i])/((predictions['PriorL'][i]*predictions['S'][i])+((1-predictions['PriorL'][i])*(1-predictions['G'][i])))
        else:
            cond_prob=(predictions['PriorL'][i]*(1-predictions['S'][i]))/((predictions['PriorL'][i]*(1-predictions['S'][i]))+((1-predictions['PriorL'][i])*predictions['G'][i]))
        
        PostL=cond_prob+((1-cond_prob)*predictions['T'][i])

        previous_student=predictions['student'][i]
        previous_skill=predictions['skill'][i]

    predictions['Prob_correct']=(predictions['PriorL']*(1-predictions['S']))+((1-predictions['PriorL'])*predictions['G'])

    return predictions

get collapsed evaluation metrics

In [5]:
def collapsed_metrics(temp_predicted_probability):
    rmse_c=sk.mean_squared_error(temp_predicted_probability['right'],temp_predicted_probability['Prob_correct'],squared=False)
    auc_c=sk.roc_auc_score(temp_predicted_probability['right'],temp_predicted_probability['Prob_correct'])
    temp=temp_predicted_probability[((temp_predicted_probability['right']==1) & (temp_predicted_probability['Prob_correct']>=0.5)) | ((temp_predicted_probability['right']==0) & (temp_predicted_probability['Prob_correct']<0.5))]
    acc_c=len(temp)/len(temp_predicted_probability)
    return rmse_c, auc_c, acc_c

get non-collapsed evaluation metrics

In [6]:
def non_collapsed_metrics(temp_predicted_probability):
    student_count=0
    auc_nc=0
    rmse_nc=0
    acc_nc=0
    all_students=temp_predicted_probability.student.unique()
    num_of_students=len(all_students)
    for i in range(num_of_students):
        temp_prediction=temp_predicted_probability[temp_predicted_probability['student']==all_students[i]]
        try:
            auc_nc=auc_nc+sk.roc_auc_score(temp_prediction['right'],temp_prediction['Prob_correct'])
            student_count=student_count+1
        except ValueError:
            pass
        rmse_nc=rmse_nc+sk.mean_squared_error(temp_prediction['right'],temp_prediction['Prob_correct'], squared=False)
        temp=temp_prediction[((temp_prediction['right']==1) & (temp_prediction['Prob_correct']>=0.5)) | ((temp_prediction['right']==0) & (temp_prediction['Prob_correct']<0.5))]
        acc_nc=acc_nc+(len(temp)/len(temp_prediction))

    auc_nc=auc_nc/student_count
    rmse_nc=rmse_nc/num_of_students
    acc_nc=acc_nc/num_of_students

    return rmse_nc, auc_nc, acc_nc

student stratified n-fold cross-validation

In [7]:
def ss_nfold_crossV(data,nfold):
    fold_rmse_c=[0]*nfold
    fold_auc_c=[0]*nfold
    fold_acc_c=[0]*nfold
    fold_rmse_nc=[0]*nfold
    fold_auc_nc=[0]*nfold
    fold_acc_nc=[0]*nfold
    avg_rmse_c=0
    avg_auc_c=0
    avg_acc_c=0
    avg_rmse_nc=0
    avg_auc_nc=0
    avg_acc_nc=0
    students=data.student.unique()
    random.shuffle(students)
    num_of_students=len(students)
    each_fold=num_of_students//nfold
    data=data[data['student'].isin(students[0:nfold*each_fold])]
    for i in range(nfold):
        testing_data=data[data['student'].isin(students[(i*each_fold):(i+1)*each_fold])]
        training_data=data[~data['student'].isin(students[(i*each_fold):(i+1)*each_fold])]
        bkt_parmtrs_4_all_skills=get_parmtrs_all_skills(training_data)
        if(len(bkt_parmtrs_4_all_skills))==0:
            fold_rmse_c[i]=fold_auc_c[i]=fold_acc_c[i]=0
            fold_rmse_nc[i]=fold_auc_nc[i]=fold_acc_nc[i]=0
        else:
            predicted_probability=get_predictions(testing_data, bkt_parmtrs_4_all_skills)
            fold_rmse_c[i], fold_auc_c[i], fold_acc_c[i]=collapsed_metrics(predicted_probability)
            fold_rmse_nc[i], fold_auc_nc[i], fold_acc_nc[i]=non_collapsed_metrics(predicted_probability)
    avg_rmse_c=statistics.mean(fold_rmse_c)
    avg_auc_c=statistics.mean(fold_auc_c)
    avg_acc_c=statistics.mean(fold_acc_c)
    avg_rmse_nc=statistics.mean(fold_rmse_nc)
    avg_auc_nc=statistics.mean(fold_auc_nc)
    avg_acc_nc=statistics.mean(fold_acc_nc)

    colpsd_rmse_auc_acc=[avg_rmse_c, avg_auc_c, avg_acc_c]
    non_colpsd_rmse_auc_acc=[avg_rmse_nc, avg_auc_nc, avg_acc_nc]
    clpsd_fold_rmse_auc_acc=[fold_rmse_c, fold_auc_c, fold_acc_c]
    non_clpsd_fold_rmse_auc_acc=[fold_rmse_nc, fold_auc_nc, fold_acc_nc]

    return colpsd_rmse_auc_acc, non_colpsd_rmse_auc_acc, clpsd_fold_rmse_auc_acc, non_clpsd_fold_rmse_auc_acc


In [8]:
libObj=ct.cdll.LoadLibrary("/Users/nabilch/Desktop/bkt_c/find_par.so")
# c_rmse_auc_acc, nc_rmse_auc_acc, fold_c_rmse_auc_acc, fold_nc_rmse_auc_acc=ss_nfold_crossV(df,5)
bkt_parmtrs_4_all_skills=get_parmtrs_all_skills(df)
# predicted_probability=get_predictions(df, bkt_parmtrs_4_all_skills)
# C_RMSE_AUC_ACC=collapsed_metrics(predicted_probability)
# NC_RMSE_AUC_ACC=non_collapsed_metrics(predicted_probability)


In [10]:
#debug
df_test=df[df['skill']=="g1.m2.1.1.1"]
libObj=ct.cdll.LoadLibrary("/Users/nabilch/Desktop/bkt_c/find_par.so")
bkt_parmtrs=get_parmtrs_all_skills(df_test)

testing whether removing rori students who did not attempt many qns (<10) makes any difference

In [ ]:
y=df.student.value_counts().to_frame('count').reset_index()
e=y[y['count']>9]
e.columns=['a','b']
df_test=df[df['student'].isin(e['a'])]
# libObj=ct.cdll.LoadLibrary("/Users/nabilch/Desktop/bkt_c/find_par.so")
# c_rmse_auc_acc, nc_rmse_auc_acc, fold_c_rmse_auc_acc, fold_nc_rmse_auc_acc=ss_nfold_crossV(df_test,5)

In [9]:
bkt_parmtrs_4_all_skills.to_csv("/Users/nabilch/Desktop/bkt_c/rori_rising_bkt_par.csv", index=False)